In [37]:
%load_ext autoreload
%autoreload 2

import json
import pandas as pd

from importlib.resources import files

DATA_DIR = files("biobank_olink.data")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
from itertools import chain

results = [json.load(p.open()) for p in (DATA_DIR / "results").iterdir() if p.name.startswith('two_extremes_exp')]
results = pd.DataFrame(chain.from_iterable(results))
results

,auc_score,learning_rate,n_estimators,max_depth,min_child_weight,subsample,colsample_bytree,colsample_bylevel,colsample_bynode,gamma,...,nan_handling,corr_handling,outer_splits,inner_splits,n_trials,optuna_n_workers,num_gpus,seed,study_name,feat_importance
0,0.800524,0.017620,1881,4,10,0.342886,0.792910,0.684067,0.635783,1.410117,...,remove,ignore,5,5,100,1,8,42,two_extremes_exp_sbp_th0.35_nan_remove_corr_ig...,"{'ACY1': 0.0028325922321528196, 'ERBB2': 0.002..."
1,0.809461,0.034799,2000,3,8,0.713941,0.477220,0.676861,0.484628,3.032847,...,remove,ignore,5,5,100,1,8,42,two_extremes_exp_sbp_th0.35_nan_remove_corr_ig...,"{'ERBB2': 0.003207206493243575, 'IL1R2': 0.002..."
2,0.802329,0.045651,1726,2,3,0.410506,0.702073,0.981524,0.539808,3.838577,...,remove,ignore,5,5,100,1,8,42,two_extremes_exp_sbp_th0.35_nan_remove_corr_ig...,"{'ERBB2': 0.00311840046197176, 'ACY1': 0.00279..."
3,0.792024,0.034820,1723,3,18,0.683438,0.641167,0.262548,0.805813,1.517563,...,remove,ignore,5,5,100,1,8,42,two_extremes_exp_sbp_th0.35_nan_remove_corr_ig...,"{'ERBB2': 0.003437293926253915, 'ERBB3': 0.003..."
4,0.810232,0.038071,1625,3,18,0.621316,0.415702,0.912701,0.547059,1.110242,...,remove,ignore,5,5,100,1,8,42,two_extremes_exp_sbp_th0.35_nan_remove_corr_ig...,"{'ERBB2': 0.0035992837511003017, 'ACY1': 0.003..."
5,0.802212,0.040004,1931,2,14,0.456802,0.792341,0.573024,0.599646,6.876193,...,ignore,drop,5,5,100,1,8,42,two_extremes_exp_sbp_th0.35_nan_ignore_corr_dr...,"{'ACY1': 0.0034462399780750275, 'MME': 0.00310..."
6,0.800046,0.016846,1939,5,18,0.460557,0.952461,0.912215,0.237589,0.026845,...,ignore,drop,5,5,100,1,8,42,two_extremes_exp_sbp_th0.35_nan_ignore_corr_dr...,"{'REN': 0.002530044876039028, 'ERBB2': 0.00237..."
7,0.789975,0.022747,2000,3,8,0.435023,0.796488,0.905048,0.372439,1.319777,...,ignore,drop,5,5,100,1,8,42,two_extremes_exp_sbp_th0.35_nan_ignore_corr_dr...,"{'ERBB2': 0.0023274747654795647, 'GGT1': 0.002..."
8,0.797427,0.037556,1674,2,14,0.558305,0.762818,0.885351,0.449748,2.433559,...,ignore,drop,5,5,100,1,8,42,two_extremes_exp_sbp_th0.35_nan_ignore_corr_dr...,"{'IL1R2': 0.004872176796197891, 'ERBB2': 0.004..."
9,0.798845,0.125097,1837,1,19,0.675048,0.761054,0.616416,0.664589,0.639475,...,ignore,drop,5,5,100,1,8,42,two_extremes_exp_sbp_th0.35_nan_ignore_corr_dr...,"{'ERBB3': 0.0184722188860178, 'CPM': 0.0151787..."


In [40]:
results.groupby(["target", "nan_handling", "corr_handling"]).auc_score.mean()

target  nan_handling  corr_handling
DBP     remove        drop             0.789886
SBP     ignore        drop             0.797701
        remove        drop             0.796922
                      ignore           0.802914
Name: auc_score, dtype: float64